In [1]:
!pip install wikipedia
import wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=9ab3baffb75a78b35d4acd5d61a5e398c86aa04cd900f3f70c086abb1a9a8827
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input,Conv1D,MaxPooling1D,Dense,GlobalMaxPooling1D,Embedding,SimpleRNN, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import gensim
from sklearn.metrics import confusion_matrix,classification_report
import seaborn as sns
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
import random

# nltk.download('wordnet', "/kaggle/working/nltk_data/")
# nltk.download('omw-1.4', "/kaggle/working/nltk_data/")
# ! unzip /kaggle/working/nltk_data/corpora/wordnet.zip -d /kaggle/working/nltk_data/corpora
# ! unzip /kaggle/working/nltk_data/corpora/omw-1.4.zip -d /kaggle/working/nltk_data/corpora
# nltk.data.path.append("/kaggle/working/nltk_data/")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
# Fetch entire articles
full_article1 = wikipedia.page("Artificial Intelligence").content
full_article2 = wikipedia.page("neural network").content
full_article3 = wikipedia.page("Deep learning").content
full_article4 = wikipedia.page("Machine ;earning").content
full_article5 = wikipedia.page("Data Science").content
full_article6 = wikipedia.page("Recurrent Neural Network").content
full_article7 = wikipedia.page("convolutional neural network").content

In [4]:
documents = [[full_article1], [full_article2], [full_article3], [full_article4], [full_article5], [full_article6], [full_article7]]
documents

[['Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and uses learning and intelligence to take actions that maximize their chances of achieving defined goals. Such machines may be called AIs.\nAI technology is widely used throughout industry, government, and science. Some high-profile applications include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applications, of

In [23]:
def preprocess(documents):
    processed_documents = []
    for document in documents:
        processed_doc = str(document)
        processed_doc = re.sub(r'[^a-zA-Z\s]', '', processed_doc)
        processed_doc = processed_doc.lower()

        sentaces_minus_sw = []
        # stop_words = stopwords.words('english')
        processed_doc = processed_doc.split()
        processed_doc = [sentaces_minus_sw.append(word) for word in processed_doc]
        processed_doc = ' '.join(sentaces_minus_sw)

        lemmatizer = WordNetLemmatizer()
        processed_doc = processed_doc.split()
        processed_doc = [lemmatizer.lemmatize(w) for w in processed_doc]
        processed_doc = ' '.join(processed_doc)

        processed_documents.append(processed_doc)

    return processed_documents

In [24]:
processed_docs = preprocess(documents)

In [25]:
processed_docs

['artificial intelligence ai in it broadest sense is intelligence exhibited by machine particularly computer system it is a field of research in computer science that develops and study method and software that enable machine to perceive their environment and us learning and intelligence to take action that maximize their chance of achieving defined goal such machine may be called aisnai technology is widely used throughout industry government and science some highprofile application include advanced web search engine eg google search recommendation system used by youtube amazon and netflix interacting via human speech eg google assistant siri and alexa autonomous vehicle eg waymo generative and creative tool eg chatgpt and ai art and superhuman play and analysis in strategy game eg chess and go however many ai application are not perceived a ai a lot of cutting edge ai ha filtered into general application often without being called ai because once something becomes useful enough and c

# **Word-Level prediction**

In [26]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(processed_docs)
sequences = tokenizer.texts_to_sequences(processed_docs)
word_index = tokenizer.word_index

In [27]:
len(word_index)

6363

In [28]:
max_sequence_len = 50

def prepare_sequences(sequence, max_seq_length):
    input_sequences = []
    next_words = []
    for i in range(0, len(sequence) - max_seq_length):
        input_sequences.append(sequence[i:i + max_seq_length])
        next_words.append(sequence[i + max_seq_length])
    return input_sequences, next_words

# Process each sequence for the entire document set
word_sequences = []
next_words = []
for doc_seq in sequences:
    if len(doc_seq) > 50:  # Ensure the sequence is long enough to process
        inputs, nexts = prepare_sequences(doc_seq, 50)
        word_sequences.extend(inputs)
        next_words.extend(nexts)

In [29]:
input_sequences = np.array(pad_sequences(word_sequences, maxlen=max_sequence_len, padding='pre'))

In [30]:
X = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')
y = to_categorical(next_words, num_classes=len(word_index) + 1)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [32]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(25805, 50)
(17204, 50)
(25805, 6364)
(17204, 6364)


In [33]:
num_words = len(word_index) + 1

# Define the RNN model
model = Sequential([
    Embedding(input_dim=num_words, output_dim=100, input_length=max_sequence_len),
    SimpleRNN(100, return_sequences=True),
    SimpleRNN(100, return_sequences=True),
    SimpleRNN(100),
    Dense(num_words, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 50, 100)           636400    
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 50, 100)           20100     
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 50, 100)           20100     
                                                                 
 simple_rnn_6 (SimpleRNN)    (None, 100)               20100     
                                                                 
 dense_2 (Dense)             (None, 6364)              642764    
                                                                 
Total params: 1339464 (5.11 MB)
Trainable params: 1339464 (5.11 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [41]:
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)

Epoch 1/50
503/503 [==============================] - 50s 93ms/step - loss: 8.0751 - accuracy: 0.0280 - val_loss: 7.9982 - val_accuracy: 0.0289
Epoch 2/50
503/503 [==============================] - 44s 88ms/step - loss: 7.5264 - accuracy: 0.0365 - val_loss: 8.1083 - val_accuracy: 0.0404
Epoch 3/50
503/503 [==============================] - 48s 96ms/step - loss: 7.2923 - accuracy: 0.0486 - val_loss: 7.9644 - val_accuracy: 0.0484
Epoch 4/50
503/503 [==============================] - 44s 88ms/step - loss: 6.9170 - accuracy: 0.0554 - val_loss: 8.2345 - val_accuracy: 0.0502
Epoch 5/50
503/503 [==============================] - 46s 91ms/step - loss: 6.5033 - accuracy: 0.0637 - val_loss: 8.4592 - val_accuracy: 0.0519
Epoch 6/50
503/503 [==============================] - 47s 94ms/step - loss: 6.0849 - accuracy: 0.0752 - val_loss: 8.7281 - val_accuracy: 0.0502
Epoch 7/50
503/503 [==============================] - 45s 89ms/step - loss: 5.6792 - accuracy: 0.0904 - val_loss: 8.7932 - val_accuracy:

In [42]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy*100)

336/336 [==============================] - 8s 24ms/step - loss: 14.5449 - accuracy: 0.0493
Test Accuracy: 4.934241250157356


In [34]:
num_words = len(word_index) + 1

# Define the RNN model
model2 = Sequential([
    Embedding(input_dim=num_words, output_dim=20, input_length=max_sequence_len),
    BatchNormalization(),
    # Dropout(0.5),
    SimpleRNN(25),
    Dropout(0.5),
    BatchNormalization(),
    Dense(num_words, activation='softmax')
])
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 50, 20)            127280    
                                                                 
 batch_normalization_2 (Bat  (None, 50, 20)            80        
 chNormalization)                                                
                                                                 
 simple_rnn_7 (SimpleRNN)    (None, 25)                1150      
                                                                 
 dropout_1 (Dropout)         (None, 25)                0         
                                                                 
 batch_normalization_3 (Bat  (None, 25)                100       
 chNormalization)                                                
                                                                 
 dense_3 (Dense)             (None, 6364)             

In [35]:
model2.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)

Epoch 1/50
807/807 [==============================] - 69s 82ms/step - loss: 7.5964 - accuracy: 0.0611 - val_loss: 6.9362 - val_accuracy: 0.0881
Epoch 2/50
807/807 [==============================] - 44s 55ms/step - loss: 6.5735 - accuracy: 0.0899 - val_loss: 6.8961 - val_accuracy: 0.0972
Epoch 3/50
807/807 [==============================] - 45s 56ms/step - loss: 6.3324 - accuracy: 0.1019 - val_loss: 6.9108 - val_accuracy: 0.1070
Epoch 4/50
807/807 [==============================] - 44s 54ms/step - loss: 6.1419 - accuracy: 0.1147 - val_loss: 6.9350 - val_accuracy: 0.1167
Epoch 5/50
807/807 [==============================] - 43s 53ms/step - loss: 5.9713 - accuracy: 0.1272 - val_loss: 6.9771 - val_accuracy: 0.1242
Epoch 6/50
807/807 [==============================] - 42s 52ms/step - loss: 5.8281 - accuracy: 0.1365 - val_loss: 7.0123 - val_accuracy: 0.1289
Epoch 7/50
807/807 [==============================] - 45s 56ms/step - loss: 5.6948 - accuracy: 0.1445 - val_loss: 7.0654 - val_accuracy:

In [18]:
loss, accuracy = model2.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy*100)

336/336 [==============================] - 3s 8ms/step - loss: 11.2631 - accuracy: 0.0814
Test Accuracy: 8.142897486686707


In [60]:
num_words = len(word_index) + 1

# Define the RNN model
model3 = Sequential([
    Embedding(input_dim=num_words, output_dim=30, input_length=max_sequence_len),
    SimpleRNN(50, return_sequences=True),
    SimpleRNN(10),
    Dense(num_words, activation='softmax')
])
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 50, 30)            187710    
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 50, 50)            4050      
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 10)                610       
                                                                 
 dense_2 (Dense)             (None, 6257)              68827     
                                                                 
Total params: 261197 (1020.30 KB)
Trainable params: 261197 (1020.30 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model3.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)

In [62]:
loss, accuracy = model3.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy*100)

336/336 [==============================] - 4s 12ms/step - loss: 10.4323 - accuracy: 0.0362
Test Accuracy: 3.619065508246422


In [19]:
# Function to randomly select a starting word from the test set
def get_random_start_word(processed_docs):
    all_words = ' '.join(processed_docs).split()
    return random.choice(all_words)  # Randomly select one word

In [20]:
def predict_next_words_single(model, start_word, num_prediction_steps, tokenizer, max_sequence_len):

    # Initialize the sequence with the start word
    current_sequence = start_word
    predicted_text = start_word

    for _ in range(num_prediction_steps):
        # Tokenize the current sequence
        sequence_tokens = tokenizer.texts_to_sequences([current_sequence])[0]

        # Pad the sequence to the required length
        padded_sequence_tokens = pad_sequences([sequence_tokens], maxlen=max_sequence_len, padding='pre')

        # Predict the next word index
        predicted_index = np.argmax(model.predict(padded_sequence_tokens), axis=-1)[0]

        # Convert the index back to a word
        predicted_word = tokenizer.index_word[predicted_index]

        # Append the predicted word to the predicted text
        predicted_text += " " + predicted_word

        # Update the current sequence with the new word
        current_sequence += " " + predicted_word

    return predicted_text

In [21]:
# Randomly select a starting word from the processed test documents
start_word = get_random_start_word(processed_docs)
print(f"Randomly selected starting word: '{start_word}'")

Randomly selected starting word: 'knowledge'


In [22]:
# Predict the next 10 words starting from the randomly selected word
predicted_text = predict_next_words_single(model2, start_word, 10, tokenizer, max_sequence_len)

print(f"Starting with '{start_word}', predicted text: {predicted_text}")

1/1 [==============================] - 0s 19ms/step
Starting with 'knowledge', predicted text: knowledge problem data science important term time step called rule applied


# **Character-Level prediction**

In [ ]:
# chars = sorted(list(set("".join(processed_docs))))
# char_to_index = {char: idx for idx, char in enumerate(chars)}
# index_to_char = {idx: char for idx, char in enumerate(chars)}
# num_chars = len(chars)

In [ ]:
# max_sequence_len = 1000

# def prepare_sequences(text):
#     sequences = []
#     next_chars = []
#     for i in range(0, len(text) - max_sequence_len, 1):
#         seq = text[i:i + max_sequence_len]
#         next_char = text[i + max_sequence_len]
#         sequences.append([char_to_index[char] for char in seq])
#         next_chars.append(char_to_index[next_char])
#     return sequences, next_chars

# # Prepare sequences for each document
# chars_sequences = []
# next_chars_list = []
# for doc in processed_docs:
#     sequences, next_chars = prepare_sequences(doc)
#     chars_sequences.extend(sequences)
#     next_chars_list.extend(next_chars)

In [ ]:
# sequences_padded = pad_sequences(chars_sequences, maxlen=max_sequence_len, padding='pre')
# X = np.array(sequences_padded)
# y = to_categorical(next_chars_list, num_classes=num_chars)

In [ ]:
# X.shape

In [ ]:
# y.shape

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [ ]:
# print(X_train.shape)
# print(X_test.shape)
# print(y_train.shape)
# print(y_test.shape)

In [ ]:
# # Define the RNN model
# char_model1 = Sequential([
#     Embedding(input_dim=num_chars, output_dim=100, input_length=max_sequence_len),
#     SimpleRNN(100),
#     Dense(num_chars, activation='softmax')
# ])
# char_model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# char_model1.summary()

In [ ]:
# char_model1.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

In [ ]:
# loss, accuracy = char_model1.evaluate(X_test, y_test)
# print("Test Accuracy:", accuracy*100)

In [ ]:
# # Define the RNN model
# char_model2 = Sequential([
#     Embedding(input_dim=num_chars, output_dim=300, input_length=max_sequence_len),
#     SimpleRNN(128),
#     Dense(num_chars, activation='softmax')
# ])
# char_model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# char_model2.summary()

In [ ]:
# char_model2.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

In [ ]:
# loss, accuracy = char_model2.evaluate(X_test, y_test)
# print("Test Accuracy:", accuracy*100)